In [13]:
%reload_ext autoreload
%autoreload 2

import warnings

import polars as pl

warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=FutureWarning)

_ = pl.Config.set_tbl_rows(10)
_ = pl.Config.set_tbl_cols(20)
_ = pl.Config.set_fmt_str_lengths(200)

In [14]:
from sdpc.data import (
    joined_train_df,
    test_data_df,
)

train_df: pl.DataFrame = joined_train_df()
test_df: pl.DataFrame = test_data_df()

In [15]:
# Load the provisional sybil list from external data
sybil_list_df = pl.read_parquet("../data/external/lz_provisional_sybil_list.parquet")

In [16]:
train_df = train_df.join(sybil_list_df, on="address", how="inner")
test_df = test_df.join(sybil_list_df, on="address", how="inner")

In [17]:
train_df

address,label,source,cluster
str,"decimal[1,0]",str,str
"""0x8531d29fcb92701b2e0244b1d90818da16ab37b4""",0,"""https://commonwealth.im/layerzero/discussion/EVM""","""Cluster (1)"""
"""0x0430c3b5c66ac702617181853ce6483afcb4bf50""",0,"""https://commonwealth.im/layerzero/discussion/0x00007C89DD7243c70749D2496eb4dD0CC1111111""","""cluster_1"""
"""0x0535b968eeafd8fd89939478eac647a63f80db22""",1,"""https://commonwealth.im/layerzero/discussion/0x00007C89DD7243c70749D2496eb4dD0CC1111111""","""cluster_1"""
"""0x4cbe51c2314083aaa72fe7f310fa3d772c64cef1""",1,"""https://commonwealth.im/layerzero/discussion/0x00007C89DD7243c70749D2496eb4dD0CC1111111""","""cluster_1"""
"""0x4cbff2fef0e84403e9d85de585fe8bbf97907da1""",1,"""https://commonwealth.im/layerzero/discussion/0x00007C89DD7243c70749D2496eb4dD0CC1111111""","""cluster_1"""
…,…,…,…
"""0x19530fc28cf01ff443c26ddcedb444425edd389d""",1,"""nansen""","""177577"""
"""0x5aea801cb60c6e968d2beb2a6b87d068e889bf3d""",0,"""nansen""","""180650"""
"""0x3fd249d41d3f1444452329381aa07701cb754fcf""",1,"""nansen""","""183192"""


In [9]:
train_df.group_by(["cluster", "label"]).agg(pl.col("address").count()).sort(
    "address", descending=True
)

cluster,label,address
str,"decimal[1,0]",u32
"""chaos""",1,148
"""chaos""",0,92
"""Cluster_96""",1,65
"""cluster_1""",1,55
"""Cluster 1 """,1,48
…,…,…
"""Cluster 645""",1,1
"""Cluster 549""",1,1
"""Cluster_1085""",0,1
